In [1]:
import os
import typing
from tqdm import tqdm
import glob
import pandas as pd
import tensorflow as tf
import numpy as np
import json

from keras.utils import  Sequence
os.path.exists('/kaggle/input/drive-redused/YaCupTrain') #/kaggle/input/drive-redused/YaCupTrain

2025-12-30 14:39:33.612665: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767105573.822612      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767105573.882896      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767105574.396771      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767105574.396817      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767105574.396820      55 computation_placer.cc:177] computation placer alr

True

In [2]:
ROOT_DATA_FOLDER = r"/kaggle/input/drive-redused"

TRAIN_DATASET_PATH = r'/kaggle/input/drive-redused/YaCupTrain'

#os.path.join(ROOT_DATA_FOLDER, r"YaCupTrain")

TEST_DATASET_PATH = os.path.join(ROOT_DATA_FOLDER, r"YaCupTest")
label_columns = ['x', 'y', 'yaw']

# Load all ids of a dataset

def read_testcase_ids(dataset_path: str):
    ids = [int(case_id) for case_id in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, case_id))]
    return ids

ids = read_testcase_ids(TRAIN_DATASET_PATH)
len(ids)

train_ids = np.random.choice(ids, size=round(0.70*len(ids)), replace=False, p=None)
test_ids = [el for el in ids if el not in train_ids]


class DataFilePaths:
    def __init__(self, testcase_path: str):
        self.testcase_path = testcase_path

    def localization(self):
        return os.path.join(self.testcase_path, 'localization.csv')

    def control(self):
        return os.path.join(self.testcase_path, 'control.csv')

    def metadata(self):
        return os.path.join(self.testcase_path, 'metadata.json')

    # exists only for test_dataset
    def requested_stamps(self):
        return os.path.join(self.testcase_path, 'requested_stamps.csv')


def read_localization(localization_path: str):
    return pd.read_csv(localization_path)

def read_control(control_path):
    return pd.read_csv(control_path)

def read_metadata(metadata_path: str):
    with open(metadata_path, 'r') as f:
        data = json.load(f)
    return data

def read_requested_stamps(requested_stamps_path: str):
    return pd.read_csv(requested_stamps_path)

def read_testcase(dataset_path: str, testcase_id: str, is_test: bool = False):
    testcase_path = os.path.join(dataset_path, str(testcase_id))
    data_file_paths = DataFilePaths(testcase_path)

    testcase_data = {}
    testcase_data['localization'] = read_localization(data_file_paths.localization())
    testcase_data['control'] = read_control(data_file_paths.control())
    testcase_data['metadata'] = read_metadata(data_file_paths.metadata())
    if is_test:
        testcase_data['requested_stamps'] = read_requested_stamps(data_file_paths.requested_stamps())

    return testcase_data


def read_testcases(dataset_path: str, is_test: bool = False, testcase_ids: typing.Iterable[int] = None):
    result = {}
    if testcase_ids is None:
        testcase_ids = read_testcase_ids(dataset_path)

    for testcase_id in tqdm(testcase_ids):
        testcase = read_testcase(dataset_path, testcase_id, is_test=is_test)
        result[testcase_id] = testcase
    return result

In [3]:
train_dataset = read_testcases(TRAIN_DATASET_PATH)
len(train_dataset)

100%|██████████| 3884/3884 [02:00<00:00, 32.13it/s]


3884

In [4]:
train_dataset[0]['control']

,stamp_ns,acceleration_level,steering
0,2987440736,-114,-2.655140
1,3027341070,-123,-2.598169
2,3066793076,-132,-2.544422
3,3106757146,-141,-2.544422
4,3146784622,-147,-2.488557
...,...,...,...
1495,62786741116,33,117.135357
1496,62826899778,33,119.059706
1497,62867315073,33,120.952111
1498,62906605994,32,122.802597


In [10]:
df1['stamp_ns'][0]

np.int64(2987440736)

In [12]:
df1 = train_dataset[0]['control']
df1['delts'] =  (df1['stamp_ns'] - df1['stamp_ns'][0])/10e9
df1

,stamp_ns,acceleration_level,steering,delts
0,2987440736,-114,-2.655140,0.000000
1,3027341070,-123,-2.598169,0.003990
2,3066793076,-132,-2.544422,0.007935
3,3106757146,-141,-2.544422,0.011932
4,3146784622,-147,-2.488557,0.015934
...,...,...,...,...
1495,62786741116,33,117.135357,5.979930
1496,62826899778,33,119.059706,5.983946
1497,62867315073,33,120.952111,5.987987
1498,62906605994,32,122.802597,5.991917


In [13]:
df2 = train_dataset[0]['localization']
df2['delts'] =  (df2['stamp_ns'] - df2['stamp_ns'][0])/10e9
df2



,stamp_ns,x,y,z,roll,pitch,yaw,delts
0,0,-4292.313705,-14527.266319,66.043314,0.003926,-0.054198,-1.936810,0.000000
1,39989868,-4292.489928,-14527.726083,66.070022,0.003702,-0.054172,-1.936858,0.003999
2,79819886,-4292.662729,-14528.183063,66.090338,0.002404,-0.054628,-1.936827,0.007982
3,125154671,-4292.862032,-14528.702952,66.120814,0.002709,-0.054559,-1.936894,0.012515
4,159636974,-4293.011898,-14529.097871,66.138226,0.003264,-0.053668,-1.936876,0.015964
...,...,...,...,...,...,...,...,...
1577,63079991016,-4445.953846,-14935.006202,66.762048,0.002899,-0.001396,-1.602179,6.307999
1578,63120445325,-4445.959637,-14935.233032,66.762420,0.001984,-0.001401,-1.590317,6.312045
1579,63160340945,-4445.966875,-14935.455432,66.758763,-0.000359,-0.001176,-1.578275,6.316034
1580,63200463435,-4445.967213,-14935.680835,66.758973,0.000547,-0.000704,-1.565992,6.320046


In [7]:
train_dataset[0]['localization'].head(22)

,stamp_ns,x,y,z,roll,pitch,yaw
0,0,-4292.313705,-14527.266319,66.043314,0.003926,-0.054198,-1.936810
1,39989868,-4292.489928,-14527.726083,66.070022,0.003702,-0.054172,-1.936858
2,79819886,-4292.662729,-14528.183063,66.090338,0.002404,-0.054628,-1.936827
3,125154671,-4292.862032,-14528.702952,66.120814,0.002709,-0.054559,-1.936894
4,159636974,-4293.011898,-14529.097871,66.138226,0.003264,-0.053668,-1.936876
5,200261036,-4293.190228,-14529.562932,66.165038,0.003789,-0.053473,-1.936975
6,239886813,-4293.364739,-14530.017999,66.190968,0.004288,-0.053212,-1.937040
7,280229844,-4293.543296,-14530.485384,66.212334,0.005093,-0.053271,-1.937132
8,320020956,-4293.718550,-14530.942045,66.238342,0.006008,-0.053025,-1.937321
9,360304997,-4293.893972,-14531.406804,66.254371,0.005946,-0.052526,-1.937373


In [14]:
list_dfs = []
for k in train_dataset.keys():
    list_dfs.append(train_dataset[k]['localization'])

In [ ]:
### implementation of the metric calculation

In [17]:

class MultiDFSequence(tf.keras.utils.Sequence):

    def __init__(self, dfs, input_cols, target_cols, window_size, batch_size, shuffle=True):

        """
        dfs: list[pd.DataFrame] — каждая df отдельная траектория
        input_cols: список входных колонок
        target_cols: список target колонок
        window_size: длина входного окна
        batch_size: размер батча
        shuffle: случайный выбор окон
        """

        self.dfs = dfs
        self.input_cols = input_cols
        self.target_cols = target_cols
        self.window_size = window_size
        self.batch_size = batch_size
        self.shuffle = shuffle
        # отфильтруем слишком короткие траектории
        self.valid_dfs = [df for df in dfs if len(df) > window_size]

        if len(self.valid_dfs) == 0:
            raise ValueError("Нет df достаточной длины")



    def __len__(self):
        # приблизительное число батчей на эпоху
        return sum(len(df) for df in self.valid_dfs) // self.batch_size



    def __getitem__(self, idx):
        X, y = [], []
        while len(X) < self.batch_size:
            s_len = len(self.valid_dfs)
            s_random = np.random.choice(s_len)
            df = self.valid_dfs[s_random]
            #df = np.random.choice(self.valid_dfs)
            start = np.random.randint(0, len(df) - self.window_size )
            x_window = df.iloc[start : start + self.window_size][self.input_cols].values
            y_target = df.iloc[start + self.window_size][self.target_cols].values
            X.append(x_window)
            y.append(y_target)
        return np.array(X), np.array(y)


class MultiStepMultiDFSequence(MultiDFSequence):

    def __init__(self, *args, horizon=10, **kwargs):
        super().__init__(*args, **kwargs)
        self.horizon = horizon


    def __getitem__(self, idx):
        X, y = [], []
        while len(X) < self.batch_size:
            s_len = len(self.valid_dfs)
            s_random = np.random.choice(s_len)
            df = self.valid_dfs[s_random]
            max_start = len(df) - self.window_size - self.horizon
            if max_start <= 0:
                continue
            start = np.random.randint(0, max_start)
            X.append(df.iloc[start : start + self.window_size][self.input_cols].values)
            y.append(df.iloc[start + self.window_size : start + self.window_size + self.horizon][self.target_cols].values)
        return np.array(X), np.array(y)






In [18]:

input_cols = ['stamp_ns','x','y','z', 'roll','pitch','yaw']
target_cols = ['x','y']

seq = MultiDFSequence(
    dfs=list_dfs,
    input_cols=input_cols,
    target_cols=target_cols,
    window_size=10,
    batch_size=64)


In [27]:
len(list_dfs)

3884

In [55]:
list_dfs[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1582 entries, 0 to 1581
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   stamp_ns  1582 non-null   int64  
 1   x         1582 non-null   float64
 2   y         1582 non-null   float64
 3   z         1582 non-null   float64
 4   roll      1582 non-null   float64
 5   pitch     1582 non-null   float64
 6   yaw       1582 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 86.6 KB


In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Input((30, 4)),
    tf.keras.layers.LSTM(64, return_sequences=False),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(1)])

model.compile(
    optimizer="adam",
    loss="mse")

model.fit(seq, epochs=1)

I0000 00:00:1767086041.101017      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1767086041.104887      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


I0000 00:00:1767086044.420542     267 cuda_dnn.cc:529] Loaded cuDNN version 91002


    1/95968 ━━━━━━━━━━━━━━━━━━━━ 59:19:34 2s/step - loss: 225750176.0000<class 'list'>
    2/95968 ━━━━━━━━━━━━━━━━━━━━ 1:53:14 71ms/step - loss: 263395152.0000<class 'list'>
    3/95968 ━━━━━━━━━━━━━━━━━━━━ 1:49:11 68ms/step - loss: 258301184.0000<class 'list'>
    4/95968 ━━━━━━━━━━━━━━━━━━━━ 1:45:31 66ms/step - loss: 256466304.0000<class 'list'>
    5/95968 ━━━━━━━━━━━━━━━━━━━━ 1:45:25 66ms/step - loss: 251365376.0000<class 'list'>
    6/95968 ━━━━━━━━━━━━━━━━━━━━ 1:44:54 66ms/step - loss: 245949408.0000<class 'list'>
    7/95968 ━━━━━━━━━━━━━━━━━━━━ 1:44:42 65ms/step - loss: 241781024.0000<class 'list'>
    8/95968 ━━━━━━━━━━━━━━━━━━━━ 1:44:16 65ms/step - loss: 238175296.0000<class 'list'>
    9/95968 ━━━━━━━━━━━━━━━━━━━━ 1:43:23 65ms/step - loss: 235519024.0000<class 'list'>
   10/95968 ━━━━━━━━━━━━━━━━━━━━ 1:41:54 64ms/step - loss: 232422048.0000<class 'list'>
   11/95968 ━━━━━━━━━━━━━━━━━━━━ 1:40:42 63ms/step - loss: 230073024.0000<class 'list'>
   12/95968 ━━━━━━━━━━━━━━━━━━━━ 

KeyboardInterrupt: 